In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [34]:
def get_costos(X, d):
    fijos = 20.6 * d
    originacion = 20.42
    fondeo = (0.20/52) * X * d

    costos = fijos + originacion + fondeo + X

    #ingresos_intereses = total_pagado - X

    seguros = {4_000 : 26, 21_000 : 29, 51_000 : 30, np.inf: 31}

    ingreso_seguro = 0.0

    key_ant = 0
    for s in seguros.keys():
        if X < s and X > key_ant:
            ingreso_seguro = seguros[s]

        key_ant = s
        
    #costos -= ingreso_seguro

    #marginal = ingresos_intereses - costos

    return costos


In [4]:
def weeks_between_dates(date_str1, date_str2, cerrado, date_format='%Y-%m-%d'):
    if cerrado == 'NO' and (pd.to_datetime(date_str2) < pd.to_datetime('2024-06-18')):
        date_str2 = '2024-06-18'
        
    # Convert the string dates to datetime objects
    date1 = pd.to_datetime(date_str1, format=date_format)
    date2 = pd.to_datetime(date_str2, format=date_format)
    
    # Calculate the difference in days
    days_difference = abs((date2 - date1).days)
    
    # Convert days to weeks
    weeks_difference = np.round(days_difference / 7)
    
    return weeks_difference, date_str2


In [39]:
def excel_to_datetime(excel_date):
    # Excel date base is January 1, 1900
    base_date = pd.Timestamp('1900-01-01')
    # Convert the Excel date number to a pandas Timedelta
    delta = pd.Timedelta(days=excel_date - 2)  # Subtract 2 days: one for the Excel leap year bug and one for inclusive counting
    # Add the delta to the base date
    result_date = base_date + delta
    return result_date
"""
data_marginal = pd.read_excel("./data/240531 PERDIDA DE COSECHA.xlsb", sheet_name='COLOCACION', header = 1)
data_marginal.columns = [k.lower().replace(" ","_") for k in data_marginal.columns]
data_marginal = data_marginal[['sucursal', 'id_dist.',
       'id_asociado',  'ciclo_credito',
       'pago_quincenal', 'inicio_credito', 'fin_credito',
       'capital', 'interes', 'iva', 'seguro',  'tasa','cerrado',
       'tipo_producto', 'id_distribuidor_linea_de_credito',
       'ciclo_del_crédito','año', 'mes']]
       
data_marginal.inicio_credito = data_marginal.inicio_credito.apply(excel_to_datetime)
data_marginal.fin_credito = data_marginal.fin_credito.apply(excel_to_datetime)
data_marginal.rename(columns={"id_dist.": "id_distribuidor"}, inplace = True)

data_marginal = data_marginal[['sucursal', 'id_distribuidor',
       'id_asociado',  'ciclo_credito',
       'pago_quincenal', 'inicio_credito', 'fin_credito',
       'capital', 'interes', 'iva', 'seguro',  'tasa',
       'tipo_producto', 'id_distribuidor_linea_de_credito',
       'ciclo_del_crédito']]
data_marginal['duracion_semanas'], data_marginal['fin_credito'] = zip(*data_marginal.apply(lambda row: weeks_between_dates(row['inicio_credito'], row['fin_credito'], row['cerrado']), axis = 1))
data_marginal['costos'] = data_marginal.apply(lambda row: get_costos(row['capital'], row['duracion_semanas']), axis = 1)

data_marginal.to_csv("./data/staging/data_marginal.csv", index = False)
"""
data_marginal = pd.read_csv("./data/staging/data_marginal.csv")

In [40]:
data_marginal.head()

,sucursal,id_distribuidor,id_asociado,ciclo_credito,pago_quincenal,inicio_credito,fin_credito,capital,interes,iva,seguro,tasa,tipo_producto,id_distribuidor_linea_de_credito,cerrado,ciclo_del_crédito,duracion_semanas,costos
0,CHALCO,125565,139614,1,445,2023-01-15,2023-05-31,3000.0,1243.20,198.91,400.0,162.56,LINEA,NaN,SI,1,19.0,3631.050769
1,NEZAHUALCOYOTL,139409,139412,3,1110,2023-01-09,2023-06-15,14000.0,3239.13,518.26,464.0,132.00,PRESTAMO PERSONAL RED,115345.0,SI,3,22.0,15658.235385
2,VILLAHERMOSA,139418,139419,5,1909,2023-01-13,2023-08-15,25000.0,4766.85,762.70,480.0,110.00,PRESTAMO PERSONAL RED,93913.0,SI,5,31.0,28639.789231
3,TONALA,8514,139558,4,1060,2023-01-15,2023-04-30,6000.0,2135.31,341.65,360.0,173.60,LINEA,NaN,SI,4,15.0,6675.573846
4,CHALCO,22763,139573,9,1060,2023-01-15,2023-04-30,6000.0,2135.31,341.65,360.0,173.60,LINEA,NaN,SI,9,15.0,6675.573846


In [41]:
#data_marginal['inicio_credito'] = pd.to_datetime(data_marginal['inicio_credito'])
#data_marginal['fin_credito'] = pd.to_datetime(data_marginal['fin_credito'])

# Función para prorratear el valor por semana
def prorate_by_week(start_date, end_date, total_value):
    
    # Calcular el número total de semanas entre las fechas
    
    # Crear un rango de fechas semanal
    week_dates = pd.date_range(start=start_date, end=end_date, freq='W')
    weekly_value = total_value / (len(week_dates) -1)
    
    # Crear un DataFrame con el valor prorrateado y la fecha de corte
    prorated_data = pd.DataFrame({
        'cutoff_date': week_dates,
        'weekly_value': np.full(len(week_dates), weekly_value)
    })
    prorated_data.loc[0, "weekly_value"] = 0
    
    return prorated_data


In [42]:
data_marginal_prorrateado = pd.DataFrame([])
c = 0
for i in tqdm(range(len(data_marginal))):
    temp = prorate_by_week(data_marginal.loc[i, "inicio_credito"], data_marginal.loc[i, "fin_credito"], data_marginal.loc[i, "costos"])
    temp['id_distribuidor'] = data_marginal.loc[i, "id_distribuidor"]
    temp['sucursal'] = data_marginal.loc[i, "sucursal"]
    temp['id_asociado'] = data_marginal.loc[i, "id_asociado"]
    
    data_marginal_prorrateado = pd.concat([data_marginal_prorrateado, temp])
    c+=1
    if c == -1:
        break

100%|████████████████████████████████████████████████████████████████████████████| 28379/28379 [09:12<00:00, 51.38it/s]


In [43]:
data_marginal_prorrateado['dia'] = data_marginal_prorrateado.cutoff_date.dt.dayofyear
data_marginal_prorrateado['mes'] = data_marginal_prorrateado.cutoff_date.dt.month
data_marginal_prorrateado['semana'] = data_marginal_prorrateado.cutoff_date.dt.weekofyear

C:\Users\CIENTIFICO DE DATOS\AppData\Local\Temp\ipykernel_5224\2444572238.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  data_marginal_prorrateado['semana'] = data_marginal_prorrateado.cutoff_date.dt.weekofyear


In [44]:
data_marginal_prorrateado.to_csv("./data/staging/data_marginal_prorrateado.csv")

In [45]:
data_marginal.query("cerrado == 'NO'")#.groupby(["id_distribuidor", ""])

,sucursal,id_distribuidor,id_asociado,ciclo_credito,pago_quincenal,inicio_credito,fin_credito,capital,interes,iva,seguro,tasa,tipo_producto,id_distribuidor_linea_de_credito,cerrado,ciclo_del_crédito,duracion_semanas,costos
15117,CHALCO,81363,166405,2,395,2023-10-31,2024-06-17,4000.00,2000.06,319.91,720.0,123.11,LINEA,NaN,NO,2,33.0,5207.912308
15120,CHALCO,55544,166408,6,440,2023-10-31,2024-06-17,4500.00,2189.62,350.36,720.0,120.51,LINEA,NaN,NO,6,33.0,5771.373846
15129,NEZAHUALCOYOTL,76453,166368,5,595,2023-10-31,2024-06-17,6000.00,3034.59,485.49,720.0,124.60,LINEA,NaN,NO,5,33.0,7461.758462
15135,TEXCOCO,31465,166393,8,495,2023-10-31,2024-06-17,5000.00,2517.15,402.76,720.0,124.11,LINEA,NaN,NO,8,33.0,6334.835385
15156,CHIMALHUACAN 2,53697,166436,5,545,2023-10-31,2024-06-17,5500.00,2775.86,444.13,720.0,124.20,LINEA,NaN,NO,5,33.0,6898.296923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28374,TLAHUAC,190792,190799,1,646,2024-06-08,2024-09-21,7999.99,2013.74,322.27,576.0,142.00,PRESTAMO PERSONAL RED,190349.0,NO,1,15.0,8790.947885
28375,TLAHUAC,190792,190840,1,646,2024-06-08,2024-09-21,7999.99,2013.74,322.27,576.0,142.00,PRESTAMO PERSONAL RED,190345.0,NO,1,15.0,8790.947885
28376,TLAHUAC,190792,190841,1,646,2024-06-08,2024-09-21,7999.99,2013.74,322.27,576.0,142.00,PRESTAMO PERSONAL RED,190253.0,NO,1,15.0,8790.947885
28377,ORIZABA,75457,188975,2,880,2024-05-31,2024-10-15,6000.00,2413.75,386.20,570.0,158.03,LINEA,NaN,NO,2,20.0,6893.958462


In [70]:
data_marginal_prorrateado_cut = data_marginal_prorrateado.query("cutoff_date <= '2024-06-18'")

In [71]:
data_marginal_prorrateado.query("cutoff_date > '2024-06-18'")

,cutoff_date,weekly_value,id_distribuidor,sucursal,id_asociado,dia,mes,semana
40,2024-06-23,139.203863,139595,ORIZABA,162910,175,6,25
41,2024-06-30,139.203863,139595,ORIZABA,162910,182,6,26
42,2024-07-07,139.203863,139595,ORIZABA,162910,189,7,27
43,2024-07-14,139.203863,139595,ORIZABA,162910,196,7,28
44,2024-07-21,139.203863,139595,ORIZABA,162910,203,7,29
...,...,...,...,...,...,...,...,...
11,2024-08-18,304.012615,75965,CHIMALHUACAN 2,189053,231,8,33
12,2024-08-25,304.012615,75965,CHIMALHUACAN 2,189053,238,8,34
13,2024-09-01,304.012615,75965,CHIMALHUACAN 2,189053,245,9,35
14,2024-09-08,304.012615,75965,CHIMALHUACAN 2,189053,252,9,36


In [ ]:
75965

In [54]:
data_temporal_mes = data_marginal_prorrateado_cut.groupby("mes").weekly_value.sum().reset_index()
data_temporal_mes.rename(columns = {"weekly_value": "importe_depositado"}, inplace = True)

In [68]:
data_marginal.query("id_asociado == 189053")#.weekly_value.sum()

,sucursal,id_distribuidor,id_asociado,ciclo_credito,pago_quincenal,inicio_credito,fin_credito,capital,interes,iva,seguro,tasa,tipo_producto,id_distribuidor_linea_de_credito,cerrado,ciclo_del_crédito,duracion_semanas,costos
28378,CHIMALHUACAN 2,75965,189053,3,705,2024-05-31,2024-09-15,4000.0,1413.8,226.19,456.0,172.1,LINEA,NaN,NO,3,15.0,4560.189231


In [3]:
"""
data_pagos = pd.read_excel("./data/240531 PERDIDA DE COSECHA.xlsb", sheet_name='PAGOS DESGLOSADOS')
data_pagos.columns = [k.lower().replace(" ","_") for k in data_pagos.columns]
data_pagos['f.pago'] = data_pagos['f.pago'].apply(excel_to_datetime)
data_pagos = data_pagos[['id_distribuidor', 'id.pago', 'f.pago',
       'h.pago', 'capital', 'interes', 'iva','seguro_distribuidor', 'seguro_asociado','importe_depositado']]
       
data_pagos.rename(columns={"f.pago":"fecha_pago"}, inplace = True)
data_pagos['dia'] = pd.to_datetime(data_pagos.fecha_pago).dt.dayofyear
data_pagos['mes'] = pd.to_datetime(data_pagos.fecha_pago).dt.month
data_pagos['semana'] = pd.to_datetime(data_pagos.fecha_pago).dt.weekofyear
data_pagos.to_csv("./data/staging/data_pagos.csv", index = False)
"""

In [12]:
data_pagos = pd.read_csv("./data/staging/data_pagos.csv")

In [33]:
data_pagos.importe_depositado.sum()

338291222.17

In [17]:
data_temporal_mes

,mes,importe_depositado
0,1,1.615997e+06
1,2,1.775176e+06
2,3,2.466172e+06
3,4,2.843595e+06
4,5,2.947956e+06
5,6,3.634172e+06
6,7,3.306010e+06
7,8,2.520295e+06
8,9,2.137539e+06
9,10,2.241151e+06


In [72]:
(pd.DataFrame(data_pagos.groupby(["mes"]).importe_depositado.sum()).reset_index() - data_temporal_mes).sum()

mes                   0.000000e+00
importe_depositado    6.794953e+07
dtype: float64

In [57]:
6.794953e+07

67949530.0

In [205]:
4.646153e+07

46461530.0